# 🧠 Building an AI Agent with LangChain: A Detailed Step-by-Step Guide

Welcome! This notebook provides a detailed, beginner-friendly guide to building your first AI Agent using the LangChain framework. An AI Agent can do more than just chat; it can use **tools** to interact with the outside world, look up information, and perform actions.

### Lab Objectives:
By the end of this lab, you will have:
1.  **Set up** a Python environment with LangChain and connected it to an LLM (OpenAI).
2.  **Created and tested custom tools** for the agent (Wikipedia Search and a Calculator).
3.  **Built an Agent** that can intelligently decide which tool to use based on your question.
4.  **Added Memory** to the agent so it can remember previous parts of your conversation.
5.  **Observed the agent's reasoning process** step-by-step.

### Part 1: Setup and Configuration

#### Step 1.1: Install Required Libraries

First, we need to install the necessary Python packages. We'll add `python-dotenv` to help manage our API key.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install "langchain==0.1.16" "langchain-core==0.1.42" langchain-openai requests python-dotenv numexpr

  Using cached langchain_openai-0.3.28-py3-none-any.whl (70 kB)
  Attempting uninstall: openai
    Found existing installation: openai 2.7.1
    Uninstalling openai-2.7.1:
      Successfully uninstalled openai-2.7.1
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 1.0.2
    Uninstalling langchain-openai-1.0.2:
      Successfully uninstalled langchain-openai-1.0.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\venv\Scripts\python.exe -m pip install --upgrade pip' command.


#### Step 1.2: Set Up the OpenAI API Key

This code loads your OpenAI API key from a `.env` file located in the same directory as the notebook. 

**Instructions for local setup:**
1. Create a file named `.env` in the same folder as this notebook.
2. Add the following line to the `.env` file, replacing the placeholder with your actual key:
   `OPENAI_API_KEY="sk-YourActualKeyHere"`

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
dotenv_path = os.path.expanduser("~/dev.env")
load_dotenv(dotenv_path)

# Get the API key from the environment
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("⚠️ OpenAI API Key not found. Make sure you have a .env file in the same directory with your key.")
else:
    # Set the key for the LangChain library to use
    os.environ['OPENAI_API_KEY'] = api_key
    print("✅ OpenAI API Key loaded successfully from .env file!")

✅ OpenAI API Key loaded successfully from .env file!


### Part 2: Creating a Custom Tool

Tools are functions that an agent can use to get information or perform actions outside of its own knowledge. Our first tool will be a simple function to search Wikipedia.

#### Step 2.1: Define the Tool's Functionality

Let's create a standard Python function that takes a topic and uses the Wikipedia API to fetch a summary.

In [4]:
import requests
from urllib.parse import quote

def wiki_search(topic: str) -> str:
    """Fetches a summary from the English Wikipedia for a given topic."""
    
    base_url = "https://en.wikipedia.org/api/rest_v1/page/summary/"
    encoded_topic = quote(topic.strip())  # Encode spaces and special characters
    
    headers = {
        "Accept": "application/json",
        "User-Agent": "WikiSearchBot/1.0 (https://github.com/yourusername; contact@example.com)"
    }
    
    try:
        with requests.Session() as session:
            response = session.get(base_url + encoded_topic, headers=headers, timeout=10)
            response.raise_for_status()  # Raise for 4xx/5xx responses
            
            data = response.json()
            return data.get("extract", "No summary available.")
    
    except requests.exceptions.HTTPError as e:
        return f"HTTP error: {e.response.status_code} - {e.response.reason}"
    except requests.exceptions.Timeout:
        return "Request timed out. Please try again."
    except requests.exceptions.RequestException as e:
        return f"Error fetching data: {e}"


#### Step 2.2: Test the Raw Function

Before giving the function to our agent, let's make sure it works on its own.

In [5]:
print("Testing the wiki_search function...")
summary = wiki_search("LangChain")
print(summary)

Testing the wiki_search function...
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


#### Step 2.3: Wrap the Function as a LangChain Tool

Now, we wrap our Python function in a LangChain `Tool` object. This is how the agent recognizes it.

**This is a critical step!** The `description` field is what the agent's LLM brain reads to decide **when** to use this tool. A good description is essential for the agent to work correctly.

In [6]:
from langchain_core.tools import Tool

wiki_tool = Tool(
    name="wiki_search",
    func=wiki_search, # The function the tool will execute
    description="Use this tool to get a summary of any topic from Wikipedia. Input should be a search term."
)

**(Debugging)** Let's inspect the created tool to see its properties.

In [7]:
print(f"Tool Name: {wiki_tool.name}")
print(f"Tool Description: {wiki_tool.description}")

Tool Name: wiki_search
Tool Description: Use this tool to get a summary of any topic from Wikipedia. Input should be a search term.


### Part 3: Initializing the LLM

The agent's "brain" is a Large Language Model (LLM). We'll instantiate the OpenAI LLM wrapper from LangChain. The `temperature` parameter controls the creativity of the model's responses (0.0 is very deterministic, 1.0 is very creative).

In [8]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0.2, # Lower temperature for more factual responses
    openai_api_key=os.environ.get('OPENAI_API_KEY')
)

print("✅ LLM Initialized.")

✅ LLM Initialized.


### Part 4: Creating and Running the Agent

Now we assemble the components: the LLM (brain) and the tools (actions). We will use a **ReAct Agent**, which stands for "Reasoning and Acting." It works by thinking step-by-step to decide what to do.

Setting `verbose=True` is crucial for debugging, as it shows us the agent's chain of thought.

In [10]:
# ZERO_SHOT_REACT_DESCRIPTION: defines a type of agent that decides which tool to use based only on the tool's description. It uses the ReAct framework to reason step-by-step to find an answer.
# ZERO_SHOT: The agent doesn't need to see any examples of how to solve a problem. It figures out what to do on the fly based on your prompt and the tools it has.

from langchain.agents import initialize_agent, AgentType

# Define the list of tools our agent can use
tools = [wiki_tool]

# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True # Set to True to see the agent's thought process
)

print("✅ Agent created successfully.")

✅ Agent created successfully.


c:\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


#### Step 4.1: Test the Agent

Let's ask a question that requires looking up information. Watch the output from `verbose=True` to see how the agent thinks:
1.  **Thought:** The agent thinks about whether it needs a tool.
2.  **Action:** It decides to use the `wiki_search` tool.
3.  **Action Input:** It determines what to search for.
4.  **Observation:** It gets the result back from the tool.
5.  **Thought:** It thinks about the final answer based on the observation.
6.  **Final Answer:** It gives you the result.

In [11]:
query = "Tell me about the planet Mars."
print(f"Executing query: {query}")
agent.run(query)

Executing query: Tell me about the planet Mars.


> Entering new AgentExecutor chain...


c:\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 I should use wiki_search to get information about Mars.
Action: wiki_search
Action Input: Mars
Observation: Mars is the fourth planet from the Sun. It is also known as the "Red Planet", for its orange-red appearance. Mars is a desert-like rocky planet with a tenuous atmosphere that is primarily carbon dioxide. At the average surface level the atmospheric pressure is a few thousandths of Earth’s, atmospheric temperature ranges from −153 to 20 °C, and cosmic radiation is high. Mars retains some water, in the ground as well as thinly in the atmosphere, forming cirrus clouds, fog, frost, larger polar regions of permafrost and ice caps, but no bodies of liquid surface water. Its surface gravity is roughly a third of Earth's or double that of the Moon. It is about half the Earth in width, or twice the Moon, with a diameter of 6,779 km (4,212 mi), and has a surface area the size of all the dry land of Earth.
Thought: This is interesting information, but I want to know more about Mars.
Action

'Mars is the fourth planet from the Sun, with a desert-like rocky surface and a tenuous atmosphere primarily made up of carbon dioxide. It has a diameter of 6,779 km and a surface area the size of all the dry land on Earth.'

### Part 5: Adding Memory to the Agent

By default, the agent is **stateless**—it forgets everything after answering a question. Let's add memory so we can have a real conversation.

In [12]:
from langchain.memory import ConversationBufferMemory

# The memory_key must match the input variable in the agent's prompt
memory = ConversationBufferMemory(memory_key="chat_history")

print("✅ Memory object created.")

✅ Memory object created.


Now, we re-initialize the agent, this time passing the `memory` object. We also change the `AgentType` to one that is optimized for conversation.

In [13]:
agent_with_memory = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
    memory=memory, 
    verbose=True
)

print("✅ Agent with memory created.")

✅ Agent with memory created.


#### Step 5.1: Test the Conversational Agent

Let's test it with a follow-up question. The agent should now be able to use the context from the first question to understand the second.

In [15]:
# First question
agent_with_memory.run("Who was Ada Lovelace?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wiki_search
Action Input: Ada Lovelace
Observation: Augusta Ada King, Countess of Lovelace, also known as Ada Lovelace, was an English mathematician and writer chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation. Ada Lovelace is sometimes considered to be the first computer programmer.
Thought: Do I need to use a tool? No
AI: Ada Lovelace was a pioneering mathematician and writer who is often considered to be the first computer programmer. She worked on Charles Babbage's Analytical Engine and recognized its potential for applications beyond just calculation.

> Finished chain.


"Ada Lovelace was a pioneering mathematician and writer who is often considered to be the first computer programmer. She worked on Charles Babbage's Analytical Engine and recognized its potential for applications beyond just calculation."

In [16]:
# Follow-up question
# The agent should know that "she" refers to Ada Lovelace from the previous turn.
agent_with_memory.run("When did she live?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wiki_search
Action Input: Ada Lovelace
Observation: Augusta Ada King, Countess of Lovelace, also known as Ada Lovelace, was an English mathematician and writer chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation. Ada Lovelace is sometimes considered to be the first computer programmer.
Thought: Do I need to use a tool? No
AI: Ada Lovelace lived from December 10, 1815 to November 27, 1852.

> Finished chain.


'Ada Lovelace lived from December 10, 1815 to November 27, 1852.'

**(Debugging)** Let's inspect the memory buffer to see what the agent has remembered.

In [17]:
print(memory.buffer)

Human: Who was Ada Lovelace?
AI: Ada Lovelace was a pioneering mathematician and writer who is often considered to be the first computer programmer. She worked on Charles Babbage's Analytical Engine and recognized its potential for applications beyond just calculation.
Human: Who was Ada Lovelace?
AI: Ada Lovelace was a pioneering mathematician and writer who is often considered to be the first computer programmer. She worked on Charles Babbage's Analytical Engine and recognized its potential for applications beyond just calculation.
Human: When did she live?
AI: Ada Lovelace lived from December 10, 1815 to November 27, 1852.


### Part 6: Extending the Agent with a Calculator Tool

An agent's power comes from having multiple tools. Let's add a calculator. LLMs are notoriously bad at math, so giving them a calculator tool is a common and effective pattern.

In [18]:
from langchain.chains import LLMMathChain

# LLMMathChain is a pre-built tool that is much safer than using Python's eval()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

calc_tool = Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="Use this tool to evaluate any mathematical expression."
)

print("✅ Calculator tool created.")

✅ Calculator tool created.


Now, we create a new agent that has access to **both** the Wikipedia tool and the Calculator tool.

In [19]:
multi_tool_memory = ConversationBufferMemory(memory_key="chat_history")

tools_multi = [wiki_tool, calc_tool]

multi_tool_agent = initialize_agent(
    tools=tools_multi,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=multi_tool_memory,
    verbose=True
)

print("✅ Multi-tool agent created.")

✅ Multi-tool agent created.


#### Step 6.1: Test the Multi-Tool Agent

Let's see if the agent can correctly choose the right tool for different tasks.

In [20]:
# This should use the Calculator tool
multi_tool_agent.run("What is 128 multiplied by 7?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 128 * 7

> Entering new LLMMathChain chain...
128 * 7```text
128 * 7
```
...numexpr.evaluate("128 * 7")...

Answer: 896
> Finished chain.

Observation: Answer: 896
Thought: Do I need to use a tool? No
AI: 128 multiplied by 7 is equal to 896. Is there anything else you would like to know?

> Finished chain.


'128 multiplied by 7 is equal to 896. Is there anything else you would like to know?'

In [21]:
# This should use the wiki_search tool
multi_tool_agent.run("And who is Alan Turing?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wiki_search
Action Input: Alan Turing
Observation: Alan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science.
Thought: Do I need to use a tool? No
AI: Alan Turing was a brilliant mathematician and computer scientist who made significant contributions to the field of theoretical computer science. He is often referred to as the father of theoretical computer science and is known for his work on the Turing machine, which is considered a model of a general-purpose computer.

> Finished chain.


'Alan Turing was a brilliant mathematician and computer scientist who made significant contributions to the field of theoretical computer science. He is often referred to as the father of theoretical computer science and is known for his work on the Turing machine, which is considered a model of a general-purpose computer.'

### Conclusion & Next Steps

Congratulations! You have successfully built a conversational AI agent that can use multiple tools to answer questions and perform tasks. 

#### Key Takeaways:
1.  **Agents = LLM (Brain) + Tools (Actions)**: The core idea is to give a language model the ability to interact with external systems.
2.  **Tool Descriptions are Crucial**: The agent decides which tool to use based on the `description` you provide. Make them clear and specific.
3.  **Memory Enables Conversation**: Without memory, each query is isolated. Memory provides context for follow-up questions.
4.  **`verbose=True` is Your Best Friend**: It's essential for understanding and debugging the agent's reasoning process.